In [1]:
from influxdb import InfluxDBClient
import plotly.express as px
import pandas as pd
from datetime import datetime
from math import degrees

In [2]:
client = InfluxDBClient(host='raspberrypi.local', port=8086, username='admin', password='paulflry', database='JP')

In [3]:
heading_query = """
SELECT time, value as heading
FROM "navigation.courseOverGroundTrue"
WHERE "context" = \'vessels.urn:mrn:imo:mmsi:227761930\'
AND time > \'2022-07-17T10:15:00Z\' AND time < \'2022-07-24T23:00:00Z\'
ORDER BY time DESC
LIMIT 100
"""

In [27]:
heading_result = client.query(heading_query)

In [28]:
time = []
heading = []
for result in heading_result.get_points():
    new_time, new_heading = result['time'], degrees(result['heading'])
    time.append(new_time)
    heading.append(new_heading)

time_datetime = [datetime.strptime(point, "%Y-%m-%dT%H:%M:%S%z") for point in time]
time_timestamp = [timestamp.timestamp() for timestamp in time_datetime]

min_timestamp = min(time_timestamp)
time_deltas = [timestamp - min_timestamp for timestamp in time_timestamp]

TypeError: must be real number, not NoneType

In [14]:
df = pd.DataFrame({'time': time_datetime, 'heading': heading, 'time_delta': time_deltas})

In [15]:
df.head()

,time,heading,time_delta
0,2022-07-17 19:35:08+00:00,110.03,9999.0
1,2022-07-17 19:35:07+00:00,106.03,9998.0
2,2022-07-17 19:35:06+00:00,109.48,9997.0
3,2022-07-17 19:35:05+00:00,110.74,9996.0
4,2022-07-17 19:35:04+00:00,106.18,9995.0


In [16]:
px.line_polar(df, r='time_delta', theta='heading')